In [1]:
import pandas as pd
import json
from tqdm import tqdm

In [2]:
df = pd.read_csv("../../data/raw/train.csv", chunksize=10_000)

In [3]:
import warnings

warnings.filterwarnings("ignore", category=pd.errors.SettingWithCopyWarning)

In [4]:
for i, chunk in tqdm(enumerate(df)):
    # Filter out the ones with missing data
    chunk = chunk[chunk['MISSING_DATA'] != True]
    # Turn polyline into list
    chunk['POLYLINE'] = chunk['POLYLINE'].apply(json.loads)
    # Find polyline length
    chunk['POLYLINE_LENGTH'] = chunk['POLYLINE'].apply(lambda x: len(x))
    # Filter only ones that have length bigger than 0
    chunk = chunk[chunk['POLYLINE'].apply(lambda x: len(x) > 0)]
    # Create an index for the list
    chunk['point_index'] = chunk['POLYLINE'].apply(lambda x: list(range(len(x))))
    # Explode the DF
    chunk = chunk.explode(['POLYLINE', 'point_index'], ignore_index=True)
    # Calculate timestamp for index (every call from the list is in a 15s interval)
    chunk['TIMESTAMP'] = chunk['TIMESTAMP'] + chunk['point_index'] * 15
    # Drop the index column
    chunk = chunk.drop(columns=['point_index'], errors='ignore')
    chunk['LON'] = chunk['POLYLINE'].apply(lambda x: float(x[0]))
    chunk['LAT'] = chunk['POLYLINE'].apply(lambda x: float(x[1]))
    chunk = chunk.drop(columns=['POLYLINE'], errors='ignore')

    chunk.to_parquet(f'../../data/chunks/chunk_{i}.parquet', index=False)

172it [02:53,  1.01s/it]
